In [2]:
import pandas as pd
import numpy as np
import nltk
import spacy
import re

In [47]:
import warnings
warnings.filterwarnings('once')

In [3]:
df = pd.read_csv('~/twitter_juul/input/userTimelineData.csv',lineterminator='\n', index_col=None)

In [4]:
# getting the column with tweets text
tweets = pd.DataFrame(df.tweetText)

#### Preprocessing

In [5]:
spacy.load('en')
from spacy.lang.en import English
parser = English()

In [6]:
text = tweets.iloc[12]
text

tweetText    Donut blow your shot. 🍩🏀
Name: 12, dtype: object

In [7]:
# getting stpwords from nltk
nltk.download('stopwords')
stopword_set = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/auppal8/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df_tweet = (df['tweetText'].str.lower().str.split())

In [9]:
df_tweet[444]

['i',
 'didn’t',
 'know',
 'i',
 'needed',
 'this,',
 'but',
 'i',
 'did',
 'https://t.co/wmgwiz4ixz']

In [10]:
# removing the stop words
df_tweet = df_tweet.apply(lambda x: [item for item in x if item not in stopword_set])

In [11]:
df_tweet[444]

['didn’t', 'know', 'needed', 'this,', 'https://t.co/wmgwiz4ixz']

In [12]:
df_tweet = df_tweet.str.join(" ")

In [13]:
# pattern = re.compile('[\W_]+')--
pattern = re.compile(r'^[a-zA-Z0-9_.-]*$', flags=re.IGNORECASE)
# pattern = re.compile('([^\s\w]|_)+')
df_tweet_p = df_tweet.str.replace(pattern,"")

In [14]:
df_tweet_p = df_tweet_p.str.lstrip().str.rstrip()

In [15]:
df_tweet_ = df_tweet_p

In [16]:
df_tweet_[1344155]

'abstraction games calls better programmer pay https://t.co/n5hs535aro'

In [17]:
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
#         elif token.orth_.startswith('@'):
#             lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [18]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/auppal8/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [66]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
#     tokens = [token for token in tokens if token not in stopword_set]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [68]:
df_tweet_ = pd.DataFrame(df_tweet_)

In [69]:
len(df_tweet_)

1344182

In [70]:
# taking 100,000 tweets randomly...
df_tweet_1 = df_tweet_.sample(100000)

In [71]:
# vectorised manner
df_tweet_1['tweetText'] = df_tweet_1['tweetText'].apply(prepare_text_for_lda)

In [ ]:
text_dic_2 = df_tweet_1['tweetText'].values

In [ ]:
# add hashtags to the existing text model
## finding the data related to just hashtags:::
hashtags = df['hashtags']
hashtags = hashtags.str.replace('None','NaN',regex=True)
hashtags = hashtags.str.replace('NaN','',regex=True)
hashtags.replace('', np.nan, inplace=True)
hashtags.dropna(inplace=True)

# vectorised manner
hashtags_ = hashtags.apply(prepare_text_for_lda)

In [ ]:
hastags_dict = hashtags_.values

In [ ]:
final_dict = np.append(text_dic_2,hastags_dict)

In [ ]:
import string
ascii_chars = set(string.printable)  # speeds things up
def remove_non_ascii_prinatble_from_list(list_of_words):
    return [word for word in list_of_words 
            if all(char in ascii_chars for char in word)]

In [ ]:
# not using this -- not work on flatten for word2vec
text_dic = (list(set([a for b in df_tweet_1.tweetText.tolist() for a in b])))
text_dic_ = remove_non_ascii_prinatble_from_list(text_dic)
text_dic_ = remove_non_ascii_prinatble_from_list(text_dic)
text_dic__ = [text_dic_]

In [ ]:
# might take some time -- using above option , vetorized manner
# import random
# text_data = []
# for tweet in df_tweet_:
#     tokens = prepare_text_for_lda(tweet)
#     if random.random() > .99:
#         text_data.append(tokens)

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_dic_2)
corpus = [dictionary.doc2bow(text) for text in text_dic_2]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
# LDA for 5 topics
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=8)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)

In [ ]:
for topic in topics:
    print(topic)

In [ ]:
# with 5 topics --Dont run the cell
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)

## obs of topic modelling
## assumptions: “(1) There are a fixed number of patterns of word use, groups of terms that tend to occur together in documents: topics. 
##(2) Each document in the corpus exhibits the topics to varying degree

##obs 1
# In the graph that represents the topic modelling of all the tweets, on the topic 1 we can see 'cannabis' as the 14th relevant term 
# in the top 30 most relevant topics,... Topic 1 seem to have words related to 'vaping', 'vapefam' ,'vaper' that seem to highlight 
# the topic related to e-cigg and its habits

##obs 2
# smoke seems to be common word among topic 1 and 2(words related to smokeing, tobacco and eciggs), there placement seems to highlight common 
# similarity among the topics as well. Also topic 2 seems to mention about youth(13th relevant topic)


In [ ]:
# > 4 word size
pyLDAvis.display(lda_display)

In [65]:
pyLDAvis.display(lda_display)